In [1]:
import os
from pathlib import Path
import urllib

import datetime
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import spiceypy as spice

_BASE_URL = 'https://naif.jpl.nasa.gov/pub/naif/'
_KERNEL_ROOT = Path('../data/spice_kernels')

In [2]:
def download_kernel(file_path, base_url=_BASE_URL, base_folder=_KERNEL_ROOT):
    if isinstance(file_path, str):
        file_path = Path(file_path)

    local_path = base_folder / file_path
    url = base_url + file_path.as_posix()

    # Create necessary sub-directories in the DL_PATH direction
    local_path.parent.mkdir(parents=True, exist_ok=True)

    try:
        # If the file is not present in the download directory -> download it
        if not os.path.isfile(local_path):
            print(f"Downloading {url}", end="  ")
            # Download the file with the urllib  package
            urllib.request.urlretrieve(str(url), str(local_path))
            print("Done.")
        else:
            print(f"{base_folder / file_path} already exists!")
    except urllib.error.HTTPError as e:
        print(f"Error: \n{url} could not be found: ", e)

In [3]:
kernels = list(map(Path, [
    'generic_kernels/lsk/naif0012.tls',
    'generic_kernels/pck/pck00010.tpc',
    'pds/data/lro-l-spice-6-v1.0/lrosp_1000/data/ck/lrosc_2020255_2020259_v01.lbl',
    'pds/data/lro-l-spice-6-v1.0/lrosp_1000/data/ck/lrosc_2020255_2020259_v01.bc',
    'pds/data/lro-l-spice-6-v1.0/lrosp_1000/data/spk/lrorg_2020167_2020259_v01.bsp',
    'pds/data/lro-l-spice-6-v1.0/lrosp_1000/data/spk/de421.bsp',
    'pds/data/lro-l-spice-6-v1.0/lrosp_1000/data/fk/moon_assoc_pa.tf',
    'pds/data/lro-l-spice-6-v1.0/lrosp_1000/data/fk/moon_080317.tf',
    'pds/data/lro-l-spice-6-v1.0/lrosp_1000/data/sclk/lro_clkcor_2020259_v00.tsc',
    'pds/data/lro-l-spice-6-v1.0/lrosp_1000/data/fk/lro_frames_2012255_v02.tf',
    'pds/data/lro-l-spice-6-v1.0/lrosp_1000/data/pck/moon_pa_de421_1900_2050.bpc'
]))

In [4]:
for k in kernels:
    download_kernel(k)

spice.furnsh(map(str, list(map(lambda x: _KERNEL_ROOT / x, kernels))))

In [12]:
# convert the datetime to a string, replacing the time with midnight
DATE_TODAY = '2012-05-01T00:00:00'

# convert the utc midnight string to the corresponding ET
ET_TODAY_MIDNIGHT = spice.utc2et(DATE_TODAY)

In [13]:
MOON_POS_REL_SUN, _ = spice.spkpos(targ='MOON', et=ET_TODAY_MIDNIGHT, ref='ECLIPJ2000', abcorr='NONE', obs='SUN')
MOON_POS_REL_SUN

array([-1.14330369e+08, -9.84192704e+07, -3.08960766e+04])

In [14]:
spice.pxform("LRO_LROCWAC_VIS", "MOON_PA", et=ET_TODAY_MIDNIGHT)

SpiceNOFRAMECONNECT: 
================================================================================

Toolkit version: CSPICE66

SPICE(NOFRAMECONNECT) --

At epoch 3.8910246618548E+08 TDB (2012 MAY 01 00:01:06.185 TDB), there is insufficient information available to transform from reference frame -85621 (LRO_LROCWAC_VIS) to reference frame 31000 (MOON_PA). Frame LRO_LROCWAC_VIS could be transformed to frame -85000 (LRO_SC_BUS). The latter is a CK frame; a CK file containing data for instrument or structure -85000 at the epoch shown above, as well as a corresponding SCLK kernel, must be loaded in order to use this frame. Frame MOON_PA could be transformed to frame 1 (J2000). Failure to find required CK data could be due to one or more CK files not having been loaded, or to the epoch shown above lying within a coverage gap or beyond the coverage bounds of the loaded CK files. It is also possible that no loaded CK file has required angular velocity data for the input epoch, even if a lo